### Walls in Plan 10

In [38]:
import ifcopenshell

# Open the IFC file
ifc_file = ifcopenshell.open('../Hus28_Test.ifc')

# Retrieve all IFCWALL elements
walls = ifc_file.by_type("IfcWall")

# Create a list for walls on the floor "Plan 10"
walls_in_plan_10 = []

# Loop through all walls
for wall in walls:
    # Retrieve the container of the wall to get the story information
    if wall.ContainedInStructure:
        for container in wall.ContainedInStructure:
            # Retrieve the story
            if container.RelatingStructure:
                building_storey = container.RelatingStructure
                # Check if the name of the story is "Plan 10"
                if building_storey.Name == "Plan 10":
                    walls_in_plan_10.append(wall)

# Output the number of walls on the floor "Plan 10"
print(f"Number of walls on the floor 'Plan 10': {len(walls_in_plan_10)}")


Number of walls on the floor 'Plan 10': 138


### Check Representation Type

In [28]:
# Initialize counters for each wall type
count_arbitrary_profile_with_voids = 0
count_extruded_area_solid = 0
count_other = 0

# Loop through the filtered walls on the floor 'Plan 10'
for wall_index, wall in enumerate(walls_in_plan_10):
    # Set flags for the wall type to False
    is_arbitrary_profile_with_voids = True
    is_extruded_area_solid = True

    print(f"Checking wall {wall_index + 1}/{len(walls_in_plan_10)}: {wall.GlobalId}")

    # Get the ShapeRepresentation of the wall
    if wall.Representation:
        for rep_index, representation in enumerate(wall.Representation.Representations):
            # Check if the representation type is 'SweptSolid'
            if representation.RepresentationType == 'SweptSolid':
                print(f"  ShapeRepresentation {rep_index + 1}/{len(wall.Representation.Representations)} Type: {representation.RepresentationType}")
                for item_index, item in enumerate(representation.Items):
                    # Check the type of geometry
                    item_type = item.is_a()
                    print(f"    Layer {item_index + 1}/{len(representation.Items)} Type: {item_type}")

                    if item_type == "IfcExtrudedAreaSolid":
                        is_arbitrary_profile_with_voids = False
                    elif item_type == "IfcArbitraryProfileDefWithVoids":
                        is_extruded_area_solid = False
                    else:
                        # If there is another type of layer present
                        is_arbitrary_profile_with_voids = False
                        is_extruded_area_solid = False

    # Classify the wall based on the flags
    if is_arbitrary_profile_with_voids:
        count_arbitrary_profile_with_voids += 1
    elif is_extruded_area_solid:
        count_extruded_area_solid += 1
    else:
        count_other += 1

# Output the count of different wall types
print(f"Number of walls with IFCARBITRARYPROFILEDEFWITHVOIDS: {count_arbitrary_profile_with_voids}")
print(f"Number of walls with IFCEXTRUDEDAREASOLID: {count_extruded_area_solid}")
print(f"Number of walls with other geometries: {count_other}")


Checking wall 1/138: 0aNC3YsFzEZf8uB9g1nSdk
  ShapeRepresentation 2/2 Type: SweptSolid
    Layer 1/5 Type: IfcExtrudedAreaSolid
    Layer 2/5 Type: IfcExtrudedAreaSolid
    Layer 3/5 Type: IfcExtrudedAreaSolid
    Layer 4/5 Type: IfcExtrudedAreaSolid
    Layer 5/5 Type: IfcExtrudedAreaSolid
Checking wall 2/138: 0aNC3YsFzEZf8uB9g1nSdj
  ShapeRepresentation 2/2 Type: SweptSolid
    Layer 1/5 Type: IfcExtrudedAreaSolid
    Layer 2/5 Type: IfcExtrudedAreaSolid
    Layer 3/5 Type: IfcExtrudedAreaSolid
    Layer 4/5 Type: IfcExtrudedAreaSolid
    Layer 5/5 Type: IfcExtrudedAreaSolid
Checking wall 3/138: 0aNC3YsFzEZf8uB9g1nSdi
  ShapeRepresentation 2/2 Type: SweptSolid
    Layer 1/5 Type: IfcExtrudedAreaSolid
    Layer 2/5 Type: IfcExtrudedAreaSolid
    Layer 3/5 Type: IfcExtrudedAreaSolid
    Layer 4/5 Type: IfcExtrudedAreaSolid
    Layer 5/5 Type: IfcExtrudedAreaSolid
Checking wall 4/138: 0aNC3YsFzEZf8uB9g1nSdZ
  ShapeRepresentation 2/2 Type: SweptSolid
    Layer 1/3 Type: IfcExtrudedAreaSo

### Test to Get Data to Reconstruct Wall[0]

In [29]:
import ifcopenshell
import numpy as np
from ifcopenshell.util import placement

# Initialize a variable for the selected wall
selected_wall = None

# Loop through the filtered walls to find the first wall with IFCEXTRUDEDAREASOLID
for wall in walls_in_plan_10:
    if wall.Representation:
        for representation in wall.Representation.Representations:
            if representation.RepresentationType == 'SweptSolid':
                for item in representation.Items:
                    if item.is_a("IfcExtrudedAreaSolid"):
                        selected_wall = wall
                        break
            if selected_wall:
                break
    if selected_wall:
        break

# Check if a wall was found
if selected_wall:
    # Output the IFC-Guid of the selected wall
    print(f"IFC-Guid of the selected wall: {selected_wall.GlobalId}")

    # Find the first `IfcExtrudedAreaSolid` geometry
    first_solid = None
    for representation in selected_wall.Representation.Representations:
        if representation.RepresentationType == 'SweptSolid':
            for item in representation.Items:
                if item.is_a("IfcExtrudedAreaSolid"):
                    first_solid = item
                    break
            if first_solid:
                break

    # If an `IfcExtrudedAreaSolid` was found
    if first_solid:
        print("IfcExtrudedAreaSolid geometry found.")

        # Get the base profile of the first layer
        profile = first_solid.SweptArea
        print(f"Profile type: {profile.is_a()}")

        # Get the local placement of the wall
        local_placement = selected_wall.ObjectPlacement

        # Calculate the transformation matrix of the wall placement
        transformation_matrix = placement.get_local_placement(local_placement)

        # Check if the profile has the expected format
        if profile.is_a("IfcArbitraryClosedProfileDef"):
            outer_curve = profile.OuterCurve
            print(f"OuterCurve type: {outer_curve.is_a()}")
            if outer_curve.is_a("IfcIndexedPolyCurve"):
                print("Absolute coordinates of the profile (vertices):")
                # Get the point list from IfcIndexedPolyCurve
                point_list = outer_curve.Points
                if point_list.is_a("IfcCartesianPointList2D"):
                    for coord_pair in point_list.CoordList:
                        # The points need to be transformed into absolute coordinates in global space
                        local_point = np.array([coord_pair[0], coord_pair[1], 0.0, 1.0])  # Set Z to 0 and make it homogeneous
                        global_point = np.dot(transformation_matrix, local_point)
                        print(f"({global_point[0]}, {global_point[1]}, {global_point[2]})")
                else:
                    print("Point list is not of type IfcCartesianPointList2D.")
            else:
                print("OuterCurve is not of type IfcIndexedPolyCurve.")
        else:
            print("Profile is not of type IfcArbitraryClosedProfileDef.")
    else:
        print("No IfcExtrudedAreaSolid geometry found in the selected wall.")
else:
    print("No wall with IFCEXTRUDEDAREASOLID found.")


IFC-Guid of the selected wall: 0aNC3YsFzEZf8uB9g1nSdk
IfcExtrudedAreaSolid geometry found.
Profile type: IfcArbitraryClosedProfileDef
OuterCurve type: IfcIndexedPolyCurve
Absolute coordinates of the profile (vertices):
(73381.0, 61526.99999999979, 0.0)
(73581.0, 61526.99999999979, 0.0)
(73581.0, 62481.99999999979, 0.0)
(73381.0, 62481.99999999979, 0.0)
(73381.0, 61526.99999999979, 0.0)


### Reconstruct Wall[0]

In [35]:
import ifcopenshell
import numpy as np
from ifcopenshell.util import placement

# Funktion zur Berechnung der Rotationsmatrix um die Z-Achse
def rotation_matrix_z(angle_degrees):
    angle_radians = np.radians(angle_degrees)
    cos_a = np.cos(angle_radians)
    sin_a = np.sin(angle_radians)
    return np.array([
        [cos_a, -sin_a, 0, 0],
        [sin_a, cos_a, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

# Funktion zur Berechnung der Transformationsmatrix für IfcAxis2Placement3D
def calculate_axis2placement_transform(axis2placement):
    location = axis2placement.Location.Coordinates
    location_matrix = np.identity(4)
    location_matrix[0:3, 3] = location

    z_axis = axis2placement.Axis.DirectionRatios if axis2placement.Axis else (0.0, 0.0, 1.0)
    x_axis = axis2placement.RefDirection.DirectionRatios if axis2placement.RefDirection else (1.0, 0.0, 0.0)

    axis_matrix = placement.a2p(location, z_axis, x_axis)
    return np.dot(location_matrix, axis_matrix)

# Funktion zur Berechnung der absoluten Platzierung unter Berücksichtigung der gesamten Hierarchie
def calculate_absolute_placement(local_placement):
    total_matrix = placement.get_local_placement(local_placement)
    
    # Navigiere durch die Hierarchie der Platzierungen
    while local_placement.PlacementRelTo:
        local_placement = local_placement.PlacementRelTo
        parent_matrix = placement.get_local_placement(local_placement)
        total_matrix = np.dot(parent_matrix, total_matrix)
    
    return total_matrix

# Funktion zum Durchsuchen aller IfcShapeAspect-Instanzen, um das Material zu finden
def find_shape_aspect_material(file, extruded_area_solid):
    for aspect in file.by_type("IfcShapeAspect"):
        if aspect.ShapeRepresentations:
            for representation in aspect.ShapeRepresentations:
                if representation.Items and extruded_area_solid in representation.Items:
                    return aspect.Name
    return "Nicht gefunden"

# IFC-Datei öffnen
file = ifcopenshell.open('../Hus28_test.ifc')

# Initialisiere eine Variable für die ausgewählte Wand
selected_wall = None

# Schleife durch die gefilterten Wände, um die erste Wand mit IFCEXTRUDEDAREASOLID zu finden
for wall in walls_in_plan_10:
    if wall.Representation:
        for representation in wall.Representation.Representations:
            if representation.RepresentationType == 'SweptSolid':
                for item in representation.Items:
                    if item.is_a("IfcExtrudedAreaSolid"):
                        selected_wall = wall
                        break
            if selected_wall:
                break
    if selected_wall:
        break

# Prüfen, ob eine Wand gefunden wurde
if selected_wall:
    # IFC-Guid der ausgewählten Wand ausgeben
    print(f"IFC-Guid der ausgewählten Wand: {selected_wall.GlobalId}")

    # Finde alle `IfcExtrudedAreaSolid` Geometrien
    extruded_solids = []

    for representation in selected_wall.Representation.Representations:
        if representation.RepresentationType == 'SweptSolid':
            for item in representation.Items:
                if item.is_a("IfcExtrudedAreaSolid"):
                    extruded_solids.append(item)

    # Prüfen, ob wir mindestens eine ExtrudedAreaSolid haben
    if extruded_solids:
        # Funktion zum Verarbeiten eines einzelnen `IfcExtrudedAreaSolid`
        def process_extruded_area_solid(solid, index):
            print(f"\nVerarbeite Layer {index + 1} der Wand:")

            # Holen des Basisprofils
            profile = solid.SweptArea
            print(f"Profiltyp: {profile.is_a()}")

            # Material des Layers finden
            material_name = find_shape_aspect_material(file, solid)
            print(f"Material des Layers: {material_name}")

            # Punktkoordinaten initial auslesen
            point_list = profile.OuterCurve.Points
            if point_list.is_a("IfcCartesianPointList2D"):
                initial_points = [np.array([coord[0], coord[1], 0.0, 1.0]) for coord in point_list.CoordList]
                print("\nInitiale Punktkoordinaten:")
                for point in initial_points:
                    print(f"({point[0]}, {point[1]}, {point[2]})")

            # Rotationsmatrix basierend auf RefDirection berechnen
            ref_direction = solid.Position.RefDirection.DirectionRatios if solid.Position.RefDirection else (1.0, 0.0, 0.0)
            if ref_direction == (1, 0, 0):
                rotation_angle = 0
            elif ref_direction == (0, 1, 0):
                rotation_angle = 90
            elif ref_direction == (-1, 0, 0):
                rotation_angle = 180
            elif ref_direction == (0, -1, 0):
                rotation_angle = 270
            else:
                rotation_angle = 0  # Standardfall

            rotation_matrix = rotation_matrix_z(rotation_angle)
            print(f"\nRotationsmatrix (um {rotation_angle} Grad):")
            print(rotation_matrix)

            # Punkte nach Rotation
            rotated_points = [np.dot(rotation_matrix, point) for point in initial_points]
            print("\nPunktkoordinaten nach Rotation:")
            for point in rotated_points:
                print(f"({point[0]}, {point[1]}, {point[2]})")

            # Translationsmatrix basierend auf der Profilposition
            profile_location = solid.Position.Location.Coordinates
            translation_matrix = np.identity(4)
            translation_matrix[0:3, 3] = profile_location
            print(f"\nTranslationsmatrix (basierend auf Profilposition {profile_location}):")
            print(translation_matrix)

            # Punkte nach Translation basierend auf der Profilposition
            translated_points = [np.dot(translation_matrix, point) for point in rotated_points]
            print("\nPunktkoordinaten nach Translation (Profilposition):")
            for point in translated_points:
                print(f"({point[0]}, {point[1]}, {point[2]})")

            # Translationsmatrix basierend auf der Wandplatzierung
            wall_location = selected_wall.ObjectPlacement.RelativePlacement.Location.Coordinates
            wall_translation_matrix = np.identity(4)
            wall_translation_matrix[0:3, 3] = wall_location
            print(f"\nTranslationsmatrix (basierend auf Wandplatzierung {wall_location}):")
            print(wall_translation_matrix)

            # Endgültige Punkte nach allen Transformationen
            final_points = [np.dot(wall_translation_matrix, point) for point in translated_points]
            print("\nEndgültige Punktkoordinaten:")
            for point in final_points:
                print(f"({point[0]}, {point[1]}, {point[2]})")

        # Verarbeite alle Layer
        for index, solid in enumerate(extruded_solids):
            process_extruded_area_solid(solid, index)

    else:
        print("Keine IfcExtrudedAreaSolid Geometrie gefunden in der ausgewählten Wand.")
else:
    print("Keine Wand mit IFCEXTRUDEDAREASOLID gefunden.")


IFC-Guid der ausgewählten Wand: 0aNC3YsFzEZf8uB9g1nSdk

Verarbeite Layer 1 der Wand:
Profiltyp: IfcArbitraryClosedProfileDef
Material des Layers: Nicht gefunden

Initiale Punktkoordinaten:
(-100.00000000000325, -477.4999999999954, 0.0)
(100.00000000000325, -477.4999999999954, 0.0)
(100.00000000000325, 477.5000000000041, 0.0)
(-100.00000000000325, 477.5000000000041, 0.0)
(-100.00000000000325, -477.4999999999954, 0.0)

Rotationsmatrix (um 90 Grad):
[[ 6.123234e-17 -1.000000e+00  0.000000e+00  0.000000e+00]
 [ 1.000000e+00  6.123234e-17  0.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  1.000000e+00  0.000000e+00]
 [ 0.000000e+00  0.000000e+00  0.000000e+00  1.000000e+00]]

Punktkoordinaten nach Rotation:
(477.4999999999954, -100.00000000000328, 0.0)
(477.4999999999954, 100.00000000000323, 0.0)
(-477.5000000000041, 100.00000000000328, 0.0)
(-477.5000000000041, -100.00000000000323, 0.0)
(477.4999999999954, -100.00000000000328, 0.0)

Translationsmatrix (basierend auf Profilposition

In [37]:
import ifcopenshell
import numpy as np
from ifcopenshell.util import placement

# Funktion zur Berechnung der Rotationsmatrix um die Z-Achse
def rotation_matrix_z(angle_degrees):
    angle_radians = np.radians(angle_degrees)
    cos_a = np.cos(angle_radians)
    sin_a = np.sin(angle_radians)
    return np.array([
        [cos_a, -sin_a, 0, 0],
        [sin_a, cos_a, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]
    ])

# Funktion zur Berechnung der Transformationsmatrix für IfcAxis2Placement3D
def calculate_axis2placement_transform(axis2placement):
    location = axis2placement.Location.Coordinates
    location_matrix = np.identity(4)
    location_matrix[0:3, 3] = location

    z_axis = axis2placement.Axis.DirectionRatios if axis2placement.Axis else (0.0, 0.0, 1.0)
    x_axis = axis2placement.RefDirection.DirectionRatios if axis2placement.RefDirection else (1.0, 0.0, 0.0)

    axis_matrix = placement.a2p(location, z_axis, x_axis)
    return np.dot(location_matrix, axis_matrix)

# Funktion zur Berechnung der absoluten Platzierung unter Berücksichtigung der gesamten Hierarchie
def calculate_absolute_placement(local_placement):
    total_matrix = placement.get_local_placement(local_placement)
    
    # Navigiere durch die Hierarchie der Platzierungen
    while local_placement.PlacementRelTo:
        local_placement = local_placement.PlacementRelTo
        parent_matrix = placement.get_local_placement(local_placement)
        total_matrix = np.dot(parent_matrix, total_matrix)
    
    return total_matrix

# Funktion zur Verarbeitung eines einzelnen Layers
def process_layer(wall):
    # Holen der Produktdefinition
    product_definition_shape = wall.Representation

    # Finde alle ShapeAspects, die mit der Produktdefinition verbunden sind
    shape_aspects = [aspect for aspect in wall.HasAssociations if aspect.is_a('IfcShapeAspect')]

    for aspect in shape_aspects:
        material_name = aspect.Name if aspect.Name else "Unbekannt"
        print(f"Material des Layers: {material_name}")

        # Zugriff auf die ShapeRepresentation des Aspects
        shape_representation = aspect.ShapeRepresentations[0] if aspect.ShapeRepresentations else None
        if not shape_representation:
            continue

        for item in shape_representation.Items:
            if item.is_a("IfcExtrudedAreaSolid"):
                # Holen des Basisprofils
                profile = item.SweptArea
                print(f"Profiltyp: {profile.is_a()}")

                # Punktkoordinaten initial auslesen
                point_list = profile.OuterCurve.Points
                if point_list.is_a("IfcCartesianPointList2D"):
                    initial_points = [np.array([coord[0], coord[1], 0.0, 1.0]) for coord in point_list.CoordList]
                    print("\nInitiale Punktkoordinaten:")
                    for point in initial_points:
                        print(f"({point[0]}, {point[1]}, {point[2]})")

                # Rotationsmatrix basierend auf RefDirection berechnen
                ref_direction = item.Position.RefDirection.DirectionRatios if item.Position.RefDirection else (1.0, 0.0, 0.0)
                if ref_direction == (1, 0, 0):
                    rotation_angle = 0
                elif ref_direction == (0, 1, 0):
                    rotation_angle = 90
                elif ref_direction == (-1, 0, 0):
                    rotation_angle = 180
                elif ref_direction == (0, -1, 0):
                    rotation_angle = 270
                else:
                    rotation_angle = 0  # Standardfall

                rotation_matrix = rotation_matrix_z(rotation_angle)
                print(f"\nRotationsmatrix (um {rotation_angle} Grad):")
                print(rotation_matrix)

                # Punkte nach Rotation
                rotated_points = [np.dot(rotation_matrix, point) for point in initial_points]
                print("\nPunktkoordinaten nach Rotation:")
                for point in rotated_points:
                    print(f"({point[0]}, {point[1]}, {point[2]})")

                # Translationsmatrix basierend auf der Profilposition
                profile_location = item.Position.Location.Coordinates
                translation_matrix = np.identity(4)
                translation_matrix[0:3, 3] = profile_location
                print(f"\nTranslationsmatrix (basierend auf Profilposition {profile_location}):")
                print(translation_matrix)

                # Punkte nach Translation basierend auf der Profilposition
                translated_points = [np.dot(translation_matrix, point) for point in rotated_points]
                print("\nPunktkoordinaten nach Translation (Profilposition):")
                for point in translated_points:
                    print(f"({point[0]}, {point[1]}, {point[2]})")

                # Translationsmatrix basierend auf der Wandplatzierung
                wall_location = wall.ObjectPlacement.RelativePlacement.Location.Coordinates
                wall_translation_matrix = np.identity(4)
                wall_translation_matrix[0:3, 3] = wall_location
                print(f"\nTranslationsmatrix (basierend auf Wandplatzierung {wall_location}):")
                print(wall_translation_matrix)

                # Endgültige Punkte nach allen Transformationen
                final_points = [np.dot(wall_translation_matrix, point) for point in translated_points]
                print("\nEndgültige Punktkoordinaten:")
                for point in final_points:
                    print(f"({point[0]}, {point[1]}, {point[2]})")

# IFC-Datei öffnen
file = ifcopenshell.open('../Hus28_test.ifc')

# Schleife durch alle Wände im Stockwerk
for wall in walls_in_plan_10:
    if wall.is_a("IfcWall") and wall.Representation:
        print(f"Verarbeite Wand: {wall.GlobalId}")
        process_layer(wall)


Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdk
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdj
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdi
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdZ
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdY
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdW
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSda
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdx
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSd$
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdt
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSdq
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSc8
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nScE
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nScD
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSc2
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSc7
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSc6
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nScR
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nScV
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nScJ
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nScI
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSet
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSes
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSer
Verarbeite Wand: 0aNC3YsFzEZf8uB9g1nSeq
